# Diffusion MRI - Tractography pipeline

To generate the fibre tracts of your brain, follow the step by step instructions below.  

### STEP 0

Move to a different folder and make a local copy of the input data there

In [ ]:
%mkdir -p dwi
%cd dwi

In [ ]:
%%bash
cp /nesi/nobackup/uoa03264/Workshop/dwi/dwi* ./

### STEP 1 

Extract the first diffusion image from the whole volume

In [ ]:
%%bash
fslroi dwi.nii nodif 0 1

### STEP 2 
Strip the skull and extract the brain

In [ ]:
%%bash
bet nodif nodif_brain -m

### STEP 3 
Convert dwi scan from nifti format to mif format

In [ ]:
%%bash
mrconvert dwi.nii.gz -fslgrad dwi.bvec dwi.bval dwi.mif
mrconvert nodif.nii.gz nodif.mif

### STEP 4 
Convert mask from nifti format to mif format

In [ ]:
%%bash
mrconvert nodif_brain_mask.nii.gz mask.mif

### STEP 5
Generate whole brain tractography

In [ ]:
%%bash
dwi2response dhollander dwi.mif wm.txt gm.txt csf.txt

In [ ]:
%%time
%%bash
dwi2fod msmt_csd dwi.mif wm.txt fod_wm.mif gm.txt gm.mif csf.txt csf.mif -mask mask.mif

In [ ]:
%%time
%%bash
tckgen fod_wm.mif Whole_brain_10K.tck \
    -seed_image mask.mif \
    -mask mask.mif \
    -select 10000 \
    -cutoff 0.15 \
    -info

### STEP 6 
Visualise the Fibre orientation data

In [ ]:
%%bash
mrview nodif.mif -odf.load_sh fod_wm.mif

### STEP 7
Visualise the whole brain fibre tracts

In [ ]:
%%bash
mrview nodif.mif -tractography.load Whole_brain_1M.tck